In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor_research.bancor_simulator.protocol import BancorDapp
from bancor_research.bancor_simulator.core.settings import GlobalSettings
from decimal import Decimal
from fractions import Fraction
import pandas as pd

# Recall from an earlier chapter that we already defined the cooldown period and the exit fees as follows.
cooldown_time: int = 7
withdrawal_fee: Decimal = Decimal('0.0025')
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']
bnt_funding_limit: Decimal = Decimal('40000')
bnt_min_liquidity: Decimal = Decimal('10000')
price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[2.5],
                            'link':[15.00],
                            'eth':[2500.00],
                            'wbtc':[40000.00]})

network_fee: Decimal = Decimal('0.2')
trading_fee: Decimal = Decimal('0.01')

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorDapp(whitelisted_tokens=whitelisted_tokens,
              network_fee=network_fee,
              trading_fee=trading_fee,
              cooldown_time=cooldown_time,
              withdrawal_fee=withdrawal_fee,
              bnt_min_liquidity=bnt_min_liquidity,
              bnt_funding_limit=bnt_funding_limit,
              price_feeds=price_feeds)

v3.create_user('Alice')
v3.create_user('Bob')
v3.create_user('Charlie')

# The actual deposits.
v3.stake(tkn_amt=100, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=10000, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=100, tkn_name='wbtc', user_name='Bob')

v3.stake(tkn_amt=1, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=1, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=1, tkn_name='wbtc', user_name='Bob')

v3.dao_msig_init_pools(whitelisted_tokens)

v3.create_user('Trader')

v3.trade(swap_amount=2000, source_token='bnt',  target_token='link', user_name='Trader', timestep=0)

v3.trade(swap_amount=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestep=0)

v3.trade(swap_amount=1, source_token='eth',  target_token='wbtc', user_name='Trader', timestep=0)


def read_url(url):
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

# Committing to the Cooldown

As described above, users commit to a 7-day cooldown period before their withdrawal is processed. Importantly, the total number of TKN that can be redeemed for the pool tokens relinquished by the user is determined at the commencement of the cooldown period. Therefore, any value appreciation on the pool token (i.e. additional trading fees and/or auto-compounding rewards) during the cooldown period is effectively forfeited. However, the user may elect to cancel their withdrawal at any time, at which point the forfeited value appreciation is returned.

Consider the following situation. A user has 1,000 bnTKN tokens, valued at 2,000 TKN. After deciding to withdraw, they commit their bnTKN to the cooldown contract, and their value is recorded (x).

![](https://lh4.googleusercontent.com/KnsmWLD34_HSXk9jbPHONlDV_bPUwQWXR08oouK34pIleWM0QiRxIZ_4Jx3BMG3UHuaPIOTXJV_i_jOJ68QTmCyjm3UYxoZ-EATaWnbSqt5VHhDpeYnCe5nwo8mPi4xQGPYOpuAk)

After 7 days have passed, the user will be entitled to receive the value associated with their stake, at the time the cooldown process was started. Imagine that during this time, the accumulation of rewards and/or trading revenue caused the bnTKN valuation to appreciate.

![](https://lh3.googleusercontent.com/boZpmn4jRpc0dPKjl6_unb2uymvSHqO9TQjzy6kOPE5CMbooAh_P0ueBqNnxD_pVXNYsQuiXkopzFzielmR0EXjO5BYnYFV92BlKhxx3szo3YD_Bypr0nGHRp5FO2m-KT4oEAB90)\
At any point during, or after the cooldown, the user may elect to cancel the withdrawal process. Moreover, once the cooldown process has completed, the user has unlimited time to make a decision. However, the value of their withdrawal (x), will persist; if the user commits to the cooldown and then waits a century before confirming the withdrawal following the cooldown, the value of their withdrawal remains the same as it was when they started the process. However, the moment they cancel the withdrawal, the x value is erased, and the bnTKN tokens are returned to their wallet. The action of re-committing to the cooldown then generates a new x value, and thus the user can easily retrieve any lost fees and rewards, assuming their cooldown period was poorly timed. In the example above, the user can improve their withdrawal valuation by 4.095% if they cancel, and then begin the cooldown again.